# Gilkatón

In [1]:
from docx import Document
import re
import csv
import nltk

In [2]:
# Read sample document
document = Document('Documentos/P_IFT_161215_601_Acc.docx')

In [3]:
paragraphs = [para.text for para in document.paragraphs]

## Raw sentence preprocessing

In [4]:
# Returns each sentence where the given regex is found
# text := [string]
def ctx_sentence(para, regex):
    p = re.sub('C\.', 'C', para)
    sents = [w.strip() for w in re.split('[\.?!;]', p)]
    for s in sents:
        match = regex.search(s)
        if match:
            return re.sub('\\bC\\b', 'C.', s)
    return para

In [5]:
# articles = re.compile('(el artículo)|(los artículos)')
# ctx_sentence(paragraphs, articles)

In [6]:
re.split('\.?;', 'hahah. sijodjf.  fposjdf. oisdjf;  f jd ojd. 00?')

['hahah. sijodjf.  fposjdf. oisdjf', '  f jd ojd. 00?']

## Entidades 

In [7]:
ctx_regex = re.compile('(el|la|El|La) [A-ZÁÉÍÓÚÑ][a-záéíóúñ]+ [A-ZÁÉÍÓÚÑ][a-záéíóúñ]+.*?\)')
# ctx_regex = re.compile('(el|la|El|La) (([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+|((y|e|de(l| la| los)?)))[ ,\.;])+')

In [8]:
# Contexts
contexts = []
for para in paragraphs:
    for match in ctx_regex.finditer(para):
        contexts.append(match.group())

In [9]:
contexts

['la Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal de Telecomunicaciones (la “Comisión”)',
 'el Diario Oficial de la Federación el “Decreto por el que se reforman y adicionan diversas disposiciones de los artículos 6o., 7o., 27, 28, 73, 78, 94 y 105 de la Constitución Política de los Estados Unidos Mexicanos, en materia de telecomunicaciones” (el “Decreto de Reforma Constitucional”)',
 'el Instituto Federal de Telecomunicaciones (el “Instituto”)',
 'el Diario Oficial de la Federación el “Decreto por el que se expiden la Ley Federal de Telecomunicaciones y Radiodifusión, y la Ley del Sistema Público de Radiodifusión del Estado Mexicano; y se reforman, adicionan y derogan diversas disposiciones en materia de telecomunicaciones y radiodifusión” (el “Decreto de Ley”)',
 'el Diario Oficial de la Federación el “Estatuto Orgánico del Instituto Federal de Telecomunicaciones” (el “Estatuto Orgánico”)',
 'el Diario Oficial de la Federación los “Lineamient

In [10]:
# Entities

# Delete initial article
entities = [re.sub('el|la|El|La', '', ctx, 1).strip() for ctx in contexts]

# Delete abbreviations
entities = [re.sub('\(.*\)', '', ent).strip() for ent in entities]

# Delete potential garbage
entities = [re.split('[;,]', ent)[0] for ent in entities]

In [11]:
entities

['Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal de Telecomunicaciones',
 'Diario Oficial de la Federación el “Decreto por el que se reforman y adicionan diversas disposiciones de los artículos 6o.',
 'Instituto Federal de Telecomunicaciones',
 'Diario Oficial de la Federación el “Decreto por el que se expiden la Ley Federal de Telecomunicaciones y Radiodifusión',
 'Diario Oficial de la Federación el “Estatuto Orgánico del Instituto Federal de Telecomunicaciones”',
 'Diario Oficial de la Federación los “Lineamientos generales para el otorgamiento de concesiones a que se refiere el título cuarto de la Ley Federal de Telecomunicaciones y Radiodifusión”',
 'Concesión Única para Uso Comercial. Con fecha 9 de octubre de 2015',
 'Constitución Política de los Estados Unidos Mexicanos',
 'Ley Federal de Telecomunicaciones y Radiodifusión',
 'Concesión Única para Uso Comercial o para consolidar concesiones en una Concesión Única para Uso Comercial',
 'Res

In [12]:
# Abbreviations
abbreviations = []
context_clues = ['(a partir de)',
                 '(en lo sucesivo)',
                 '(de ahora en adelante)',
                 '(se entenderá por)',
                 '(se denominará como)']
context_clues_regex = re.compile('|'.join(context_clues))
for ctx in contexts:
    first_match = re.search('\(.*.*\)', ctx)
    if first_match:
        second_match = re.search('(\"|\“|\”|\').*(\“|\"|\”|\')', first_match.group())
        if second_match:
            abbreviations.append(second_match.group())
        else:
            trimmed = context_clues_regex.sub('', first_match.group())
            abbreviations.append(trimmed)
abbreviations = [re.sub('\"|\“|\”|\'', '', ab).strip() for ab in abbreviations]

In [13]:
abbreviations

['Comisión',
 'Decreto de Reforma Constitucional',
 'Instituto',
 'Decreto de Ley',
 'Estatuto Orgánico',
 'Lineamientos',
 'Solicitud de Transición',
 'Constitución',
 'Ley',
 '(i)',
 '(treinta)']

In [14]:
# Serialize the info into CSV
first_header = ['CONTEXTO', 'ENTIDAD', 'ABREVIATURAS']
with open('entidades.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(first_header)
    for row in zip(contexts, entities, abbreviations):
        writer.writerow(row)

## Sucesos

In [15]:
months = ['(enero)', 
          '(febrero)', 
          '(marzo)', 
          '(abril)',
          '(mayo)',
          '(junio)',
          '(julio)',
          '(agosto)',
          '(septiembre)',
          '(octubre)',
          '(noviembre)',
          '(diciembre)']
hundreds = ['cien', 
            'doscientos', 
            'trescientos',
            'cuatrocientos',
            'quinientos',
            'seiscientos',
            'setecientos',
            'ochocientos',
            'novecientos']
units = []
ones = []
day_regex = '(primero)|(veinte)|(treinta)'
num_map = {
    '(primero)': '1',
    '(veinte)': '20',
    '(treinta)': '30'
}
with open('0s.txt', 'r') as f:
    for num, line in enumerate(f):
        val1 = '({})'.format(line.strip())
        val2 = '(veinti{})'.format(line.strip())
        units.append(val1)
        units.append(val2)
        num_map[val1] = str(num + 1)
        num_map[val2] = '2{}'.format(str(num + 1))
        if line in ['uno', 'dos']:
            val3 = '(treinta y {})'.format(line.strip())
            units.append(val3)
            num_map[val3] = '3{}'.format(str(num + 1))
day_regex += '|' + '|'.join(units)
with open('1s.txt', 'r') as f:
    for num, line in enumerate(f):
        val = '({})'.format(line.strip())
        ones.append(val)
        num_map[val] = '1{}'.format(str(num + 1))
day_regex += '|' + '|'.join(ones)
day_regex = re.compile(day_regex)

In [16]:
num_map

{'(catorce)': '14',
 '(cinco)': '5',
 '(cuatro)': '4',
 '(diecinueve)': '19',
 '(dieciocho)': '18',
 '(diecisiete)': '17',
 '(dieciséis)': '16',
 '(doce)': '12',
 '(dos)': '2',
 '(nueve)': '9',
 '(ocho)': '8',
 '(once)': '11',
 '(primero)': '1',
 '(quince)': '15',
 '(seis)': '6',
 '(siete)': '7',
 '(trece)': '13',
 '(treinta)': '30',
 '(tres)': '3',
 '(uno)': '1',
 '(veinte)': '20',
 '(veinticinco)': '25',
 '(veinticuatro)': '24',
 '(veintidos)': '22',
 '(veintinueve)': '29',
 '(veintiocho)': '28',
 '(veintiseis)': '26',
 '(veintisiete)': '27',
 '(veintitres)': '23',
 '(veintiuno)': '21'}

In [17]:
def to_digits(text, mode):
    assert mode in ['day', 'month', 'year']
    if mode == 'day':
        try:
            match = day_regex.search(text)
            if match:
                t = match.group()
                conv = num_map['({})'.format(t)]
                return conv
            else:
                return text
        except KeyError:
            return text
    elif mode == 'month':
        for num, m in enumerate(months):
            month = '({})'.format(text)
            if m == month:
                mn = str(num + 1)
                if len(mn) < 2:
                    return '0{}'.format(mn)
                else:
                    return mn
        return ''
    elif mode == 'year':
        spl = text.split()
        if text.startswith('dos'):
            conv = '2'
            spl = spl[1:]
        else:
            conv = '1'
        for part in spl:
            match = day_regex.search(part)
            if match:
                case = '({})'.format(part)
                if conv and conv[-1] in ['9', '0'] and int(num_map[case]) < 10:
                    conv += '0{}'.format(num_map[case])
                else:
                    conv += num_map[case]
            elif part == 'mil':
                conv += '0'
            elif part in hundreds:
                conv += str(hundreds.index(part) + 1)
        return conv

In [18]:
def date_parser(text):
    day = ''
    month = ''
    year = ''
    splitted = [w.strip() for w in re.split('del?', text)]
    print(splitted)
    for word in splitted:
        day_match = re.search('\\b([123](o?\.?))?[0-9]\\b', word)
        if day_match:
            day = day_match.group()
        month_match = re.search('%s' % '|'.join(months), word)
        if month_match:
            month = month_match.group()
            month = to_digits(month, mode='month')
        year_match = re.search('[0-9]{4}', word)
        if year_match:
            year = year_match.group()
    if not day:
        day = to_digits(splitted[0], mode='day')
    if not year:
        year = to_digits(splitted[2], mode='year')
    return '/'.join([day, month, year])

In [19]:
# Single dates
single_date_regex = re.compile('de (%s) del?' % '|'.join(months))

ctx_dates = []
for para in paragraphs:
    match = single_date_regex.search(para)
    if match:
        ctx = ctx_sentence(para, single_date_regex)
        start = single_date_regex.search(ctx).start() - 3
        end = single_date_regex.search(ctx).end() + 5
        if start < 0:
            start = 0
        if end >= len(ctx):
            end = len(ctx)
        ctx_dates.append([ctx, date_parser(ctx[start : end])])

['30', 'octubre', '1997']
['8', 'febrero', '2002']
['1', 'marzo', '2004']
['1', 'diciembre', '2005']
['17', 'julio', '2007']
['11', 'junio', '2013']
['14', 'julio', '2014']
['4', 'septiembre', '2014']
['24', 'julio', '2015']
['9', 'octubre', '2015']
['17', 'noviembre', '2015']
['24', 'julio', 'año']
['19', 'octubre', '2015']
['17', 'noviembre', '2015']
['17', 'noviembre', '2015']
['30', 'octubre', '1997']
['14', 'julio', '2014']
['30', 'octubre', '1997']
['30', 'octubre', '1997']
['16', 'diciembre', '2015']


In [20]:
ctx_dates

[['El 30 de octubre de 1997, la Secretaría de Comunicaciones y Transportes (la “Secretaría”), otorgó en favor de Higinio Santiago Lastiri Quirós, un título de concesión para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la “Concesión”)',
  '30/10/1997'],
 ['Con oficio CFT/D06/CGST/CGTVAR/1122/2002 de fecha 8 de febrero de 2002, la entonces Dirección General de Televisión y Audio Restringidos, adscrita a la Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal de Telecomunicaciones (la “Comisión”), autorizó la ampliación de cobertura de la Concesión, hacia las localidades de Emiliano Zapata, Hidalgo y Nanacamilpa, Tlaxcala',
  '8/02/2002'],
 ['-1123 de fecha 1 de marzo de 2004, la Secretaría autorizó la cesión de derechos de la Concesión a favor de TV de Calpulalpan, S',

In [21]:
# Where
where_regex = re.compile('en ((el|la|los|las) )?(([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+|(de(l| la| los)?))[ ,\.;])+')
for cd in ctx_dates:
    match = where_regex.search(cd[0])
    if match:
        place = re.sub('en ((el|la|los|las) )?', '', match.group())
        cd.append(place)
    else:
        cd.append('')

In [22]:
ctx_dates

[['El 30 de octubre de 1997, la Secretaría de Comunicaciones y Transportes (la “Secretaría”), otorgó en favor de Higinio Santiago Lastiri Quirós, un título de concesión para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la “Concesión”)',
  '30/10/1997',
  'Calpulalpan,'],
 ['Con oficio CFT/D06/CGST/CGTVAR/1122/2002 de fecha 8 de febrero de 2002, la entonces Dirección General de Televisión y Audio Restringidos, adscrita a la Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal de Telecomunicaciones (la “Comisión”), autorizó la ampliación de cobertura de la Concesión, hacia las localidades de Emiliano Zapata, Hidalgo y Nanacamilpa, Tlaxcala',
  '8/02/2002',
  ''],
 ['-1123 de fecha 1 de marzo de 2004, la Secretaría autorizó la cesión de derechos de la Concesión a favor d

In [23]:
# In order to
to_regex = re.compile('((((para )|(a fin de )|(con el objeto de ))que)|(con la finalidad de)|(para [a-záéíóúñ]+(ar|er|ir))).+[ \.;]')
for cd in ctx_dates:
    match = to_regex.search(cd[0])
    if match:
        cd.append(match.group())
    else:
        cd.append('')

In [24]:
ctx_dates

[['El 30 de octubre de 1997, la Secretaría de Comunicaciones y Transportes (la “Secretaría”), otorgó en favor de Higinio Santiago Lastiri Quirós, un título de concesión para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la “Concesión”)',
  '30/10/1997',
  'Calpulalpan,',
  'para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la '],
 ['Con oficio CFT/D06/CGST/CGTVAR/1122/2002 de fecha 8 de febrero de 2002, la entonces Dirección General de Televisión y Audio Restringidos, adscrita a la Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal de Telecomunicaciones (la “Comisión”), autorizó la 

In [25]:
# to whom
whom_regex = re.compile('\\b(al?|(en favor de( la|l| los| las)?)) [a-záéíóú]* ?(([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚa-záéíóúñ]*|(de(l| los| la| las)?))[ ,\.;])*')
for cd in ctx_dates:
    match = whom_regex.search(cd[0])
    if match:
        cd.append(match.group())
    else:
        cd.append('')

In [26]:
ctx_dates

[['El 30 de octubre de 1997, la Secretaría de Comunicaciones y Transportes (la “Secretaría”), otorgó en favor de Higinio Santiago Lastiri Quirós, un título de concesión para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la “Concesión”)',
  '30/10/1997',
  'Calpulalpan,',
  'para instalar, operar y explotar una red pública de telecomunicaciones para prestar el servicio de televisión restringida, con cobertura en Calpulalpan, Tlaxcala con una vigencia de 30 (treinta) años contados a partir de su otorgamiento (la ',
  'en favor de Higinio Santiago Lastiri Quirós,'],
 ['Con oficio CFT/D06/CGST/CGTVAR/1122/2002 de fecha 8 de febrero de 2002, la entonces Dirección General de Televisión y Audio Restringidos, adscrita a la Coordinación General de Servicios de Telecomunicaciones de la extinta Comisión Federal d

In [34]:
# CSV serializer
suc_header = ['CONTEXTO', 'FECHAS', 'LEMA', 'QUIÉN', 'QUÉ', 'DÓNDE', 'A QUIÉN', 'PARA QUÉ']
with open('sucesos.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(suc_header)
    for data in ctx_dates:
        writer.writerow([data[0], data[1], '', '', '', data[2], data[4], data[3]])

## Leyes

In [27]:
ctx_regex3 = re.compile('((el artículo)|(los artículos)).*[\.;]')

In [28]:
# Contexts
contexts3 = []
for para in paragraphs:
    for match in ctx_regex3.finditer(para):
        splitted = match.group().split()
        ctx = ''
        for word in splitted:
            ctx += '{} '.format(word)
            if '.' in word or ';' in word:
                if not re.match('[0-9]+o[\.;]', word):
                    contexts3.append(ctx.strip())
                    break

In [29]:
contexts3

['los artículos 6o., 7o., 27, 28, 73, 78, 94 y 105 de la Constitución Política de los Estados Unidos Mexicanos, en materia de telecomunicaciones” (el “Decreto de Reforma Constitucional”), mediante el cual se creó el Instituto Federal de Telecomunicaciones (el “Instituto”), como un órgano autónomo que tiene por objeto el desarrollo eficiente de la radiodifusión y las telecomunicaciones.',
 'el artículo 28 párrafos décimo quinto, décimo sexto y décimo séptimo de la Constitución Política de los Estados Unidos Mexicanos (la “Constitución”), el Instituto es un órgano autónomo, con personalidad jurídica y patrimonio propios, que tiene por objeto el desarrollo eficiente de la radiodifusión y las telecomunicaciones, conforme a lo dispuesto por la propia Constitución y en los términos que fijen las leyes.',
 'los artículos 6o. y 7o. de la Constitución.',
 'el artículo Octavo Transitorio del Decreto de Ley señala que los actuales concesionarios podrán obtener autorización del Instituto para, ent

In [30]:
# Laws
def get_law(sentence):
    law_regex = re.compile('(de(l| la)) (([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+|(de(l| la| los)))[ ,\.;])+')
    match = law_regex.search(sentence)
    if match:
        return re.sub('(de(l| la))', '', match.group(), 1).strip()
    else:
        return ''

In [31]:
# Articles
arab_num_regex = re.compile('[1-9][0-9]{,2}((o\.)|(-[A-Z]))?')
roman_num_regex = re.compile('\\b[IVLX]+\\b')
article_stack = []
law_data = {}
frac_mode = False
for para in paragraphs:
    for word in para.split():
        match_arab = arab_num_regex.search(word)
        if match_arab:
            if frac_mode and article_stack:
                article_stack.pop()
            frac_mode = False
            number = match_arab.group()
            try:
                law_data[number]
            except KeyError:
                reg = re.compile(number)
                ctx_sent = ctx_sentence(para, reg)
                law_data[number] = ([], ctx_sent,
                                    get_law(ctx_sent))
            article_stack.append(number)
        match_roman = roman_num_regex.search(word)
        if match_roman:
            frac_mode = True
            number = match_roman.group()
            if article_stack:
                law_data[article_stack[-1]][0].append(number)

In [32]:
law_data

{'1': ([], 'Con oficio 112', ''),
 '105': ([],
  ', 27, 28, 73, 78, 94 y 105 de la Constitución Política de los Estados Unidos Mexicanos, en materia de telecomunicaciones” (el “Decreto de Reforma Constitucional”), mediante el cual se creó el Instituto Federal de Telecomunicaciones (el “Instituto”), como un órgano autónomo que tiene por objeto el desarrollo eficiente de la radiodifusión y las telecomunicaciones',
  'Constitución Política de los Estados Unidos Mexicanos,'),
 '11': ([],
  'Con fecha 11 de junio de 2013, se publicó en el Diario Oficial de la Federación el “Decreto por el que se reforman y adicionan diversas disposiciones de los artículos 6o',
  'Federación'),
 '112': (['XXIX'], 'Con oficio 112', ''),
 '12': ([],
  'así como en los artículos 1, 7, 8 y 12 del Estatuto Orgánico del Instituto Federal de Telecomunicaciones, mediante Acuerdo P/IFT/161215/601',
  'Estatuto Orgánico del Instituto Federal'),
 '13': ([],
  'y se reforman, adicionan y derogan diversas disposiciones e

In [33]:
# Data CSV serializer
law_header = ['CONTEXTO', 'LEYES', 'ARTÍCULOS', 'FRACCIONES']
with open('leyes.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(law_header)
    for art, (fracs, ctx, law) in law_data.items():
        if not fracs:
            writer.writerow([ctx, law, art, ''])
        for frac in fracs:
            writer.writerow([ctx, law, art, frac])